<a href="https://colab.research.google.com/github/AstridSerruto/Projects/blob/master/EnergyTimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Energy Timeseries Data Visualization

## 1. Import Libraries and get files

Import Libraries

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
import os
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, month_plot
from statsmodels.tsa.stattools import kpss, adfuller
from statsmodels.tsa.seasonal import STL
from pylab import rcParams
from platform import python_version

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

The dataset for this exercise is the ComEd hourly energy consumption dataset which was recorded in megawatts (MW).

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
rcParams['figure.figsize'] = (12, 8) 
sns.set_theme(palette='muted', style='whitegrid')

In [ ]:
path = '../input/hourly-energy-consumption/COMED_hourly.csv'
df = pd.read_csv(path)
print(df.shape)
print(df.dtypes)
df.head()

## 2. Review Data Contents

Lets examine the data we have.

In [ ]:
df.info()

We can see that "Datetime" has been typed incorrectly, we will change it to "Datetime64"

In [ ]:
df['Datetime'] = df.Datetime.astype('Datetime64')
print(df.dtypes)
df.head()

In [ ]:
df['Datetime'].describe(datetime_is_numeric=True)

We can see that the Timeseries runs from 01/01/2011 to 08/03/2018

# 3. Time Plot

We will plot the energy observations agaisnt the hourly datetime records. This will tell us if the data is in the proper format.

In [ ]:
ax = sns.lineplot(data=df, x='Datetime', y='COMED_MW')
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
plt.xlabel('Datetime',color='Blue')
plt.ylabel('Energy Consumption (MW)', color='Blue')
plt.title('ComEd Hourly Energy Consumption', fontsize=18, color='Blue')
plt.show()

To help with seasonal ploting, we will add a 'QUARTER" column to the Dataframe

In [ ]:
df['QUARTER'] = df['Datetime'].dt.quarter
df.head()

To make visual interpretation easier, we will use resampling to convert the timeseries to an average monthly format.

In [ ]:
df = df.resample('M', on='Datetime').mean().rename(columns={'COMED_MW':'MONTHLY_AVG'})
df['QUARTER'] = df['QUARTER'].astype(int)
df.head()

In [ ]:
ax = sns.lineplot(data=df, x='Datetime', y='MONTHLY_AVG')
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
plt.xlabel('Datetime')
plt.ylabel('Avg. Energy Consumption (MW)')
plt.title('ComEd Monthly Average Energy Consumption', fontsize=16)
plt.show()

The above is an easier to understand time plot.

## 4. Lag plot

A lag plot is used to observe the underlying properties of a timeseries. It's a type of scatter plot where a set of data from time step y(t+i) , with time t and lag i, is plotted against data from a later time step y(t). The pattern in a lag plot will show if the timeseries data is random, sinusoidal, autocorrelates, or contains outliers. These patterns also provide information on suitable models for the data.



In [ ]:
plt.figure(figsize=(12,12))

for i in range(1, 10):
    ax=plt.subplot(3, 3, i)
    # Create a lag plot for each quarter
    pd.plotting.lag_plot(df.loc[df['QUARTER'] == 1, 'MONTHLY_AVG'], lag=i, c='red', label='Q1')    
    pd.plotting.lag_plot(df.loc[df['QUARTER'] == 2, 'MONTHLY_AVG'], lag=i, c='green', label='Q2')
    pd.plotting.lag_plot(df.loc[df['QUARTER'] == 3, 'MONTHLY_AVG'], lag=i, c='navy', label='Q3')
    pd.plotting.lag_plot(df.loc[df['QUARTER'] == 4, 'MONTHLY_AVG'], lag=i, c='orange', label='Q4')
    ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    plt.legend()
    plt.title(f'Lag {i}')

plt.tight_layout()
plt.show()

The positive linear relationship in lags 3, 6, and 9 demonstrates autocorrelation at regular intervals. This suggests that the data has seasonality (non-stationary) and rules out randomness. We can further verify this with an autocorrelation plot.

## 5. Autocorrelation Plot

The autocorrelation plot is similar to the lag plot in that it also checks for randomness in the timeseries. Autocorrelations are calculated for the data at different lags. A completely random timeseries will show autocorrelations near zero for all lags.

In [ ]:
plot_acf(df['MONTHLY_AVG'], color='red')
plt.xlabel('Lags')
plt.tight_layout()
plt.show()

The plot shows some significant non-zero autocorrelations. We see an oscillation of negative and positive correlation values. A closer look reveals large autocorrelations at multiples of the seasonal frequency (3). This is another indicator of seasonality.

## 6. Partial Autocorrelation Plot

Partial autocorrelation is also a calculation for data at different lags, but with indirect correlations removed. This plot can be used to specify a regression model for the timeseries. The first autocorrelations are typically the most significant for analysis.

In [ ]:
plot_pacf(df['MONTHLY_AVG'], color='red')
plt.xlabel('Lags')
plt.tight_layout()
plt.show()

Partial autocorrelation shows only the direct relationship between a time step and its lag. The indirect relations are less significant and fall closer to zero.

When deciding on a regression model, the timeseries will need to undergo a transformation known as differencing to remove any seasonality or trend. This will make the data stationary (no trend or seasonality) and better suited for modeling.

## 7. Timeseries Differencing

The mean of a timeseries is stabilized when the difference is calculated between consecutive steps. This reduces trend and seasonality. The data becomes stationary which means its statistical properties don't change over time. The mean, variance, and covariance remain static.

In [ ]:
diff = df['MONTHLY_AVG'].diff(1).dropna()
diff.head()

In [ ]:
ax = sns.lineplot(data=diff)
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
plt.xlabel('Datetime')
plt.ylabel('Difference')
plt.title('ComEd Monthly Average Energy Consumption', fontsize=16)
plt.show()

As shown above the changed data is more stable about the mean. To confirm stationarity we'll utilize the following two tests.

Kwiatkowski-Phillips-Schmidt-Shin Test

The Kwiatkowski-Phillips-Schmidt-Shin Test (KPSS) checks for the stationarity of a timeseries by testing the null hypothesis that the data is stationary about a trend. A p-value above 0.05 indicates the data is trend-stationary. We'll set up metric tables for our differenced data and view the results.

In [ ]:
def kpss_test(timeseries):
    # Create a metrics table for kpss test
    print('KPSS Metrics:')    
    kpsstest = kpss(timeseries, regression='c', nlags='auto')
    kpss_output = pd.Series(
        kpsstest[0:3], 
        index = ['Test Statistic', 
               'p-Value', 
               'Number of Lags'
        ]
    )
    
    for key, value in kpsstest[3].items():
        kpss_output[f'Critical Value ({key})'] = value
    
    return kpss_output

kpss_test(diff)

The p-value is above 0.05, which means our differenced data is trend-stationary. The null hypothesis is accepted.

Augmented Dickey-Fuller Test

The second of our stationarity tests is the Augmented Dickey-Fuller (ADF) Test. This test assumes the data is non-stationary as the null hypothesis. A p-value below 0.05 indicates a stationary timeseries. This is opposite the KPSS test so care must be taken when making conclusions.

In [ ]:
def adf_test(timeseries):
    # Create a metrics table for adf test
    print('Dickey-Fuller Metrics:')    
    adftest = adfuller(timeseries, autolag='AIC')
    adfoutput = pd.Series(
        adftest[0:4],
        index = [
            'Test Statistic',
            'p-Value',
            'Number of Lags',
            'Number of Observations'
        ]
    )
    
    for key, value in adftest[4].items():
        adfoutput[f'Critical Value ({key})'] = value
    
    return adfoutput

adf_test(diff)

The small p-value is below 0.05, the null hypothesis is rejected. Both ADF and KPSS tests verify that the data is stationary.

## 8. Seasonal Plots

Seasonal plots are similar to time plots but with data plotted against a seasonal period. The period can be daily, monthly, or specific to a particular use case. We will plot energy data from all years against the quarterly periods.

Quarter Plot

In [ ]:
fig, ax = plt.subplots()
sns.lineplot(ax=ax, data=df, x='QUARTER', y='MONTHLY_AVG', ci=95)
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
ax.set_xticks([1,2,3,4])
ax.set_xticklabels(['Q1', 'Q2', 'Q3', 'Q4'])
ax.set_ylabel('Avg. Energy Consumption (MW)')
ax.set_xlabel('Seasonal Quarter')
ax.set_title('ComEd Monthly Average Energy Consumption (2011 - 2018)')
plt.show()

The plot shows the third quarter sees the most monthly average energy usage each year. We can plot the data against each month to investigate further.

Month Plot

In [ ]:
fig, ax = plt.subplots()
month_plot(df['MONTHLY_AVG'], ax=ax)
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
plt.title('ComEd Monthly Average Energy Consumption',fontsize=16)
ax.set_xlabel('Monthly')
plt.ylabel('Avg. Energy Consumption (MW)')
plt.show()

We can see that July is the month where peak average energy usage occurs. January sees the highest usage out of the winter months, and the lowest points are in April and October. Overall, this is a close reflection of the pattern observed in the quarter plot.

## 9. Seasonal Moving Average

A moving average is created by calculating the mean of timeseries values over a specified set of past timesteps. This has the effect of smoothing out the time plot. We'll plot a 3 month moving average that corresponds to each seasonal quarter.

In [ ]:
df['MOVING_AVG'] = df['MONTHLY_AVG'].rolling(3, min_periods=3).mean()
df.head(12)

In the MOVING_AVG column you can observe the consecutive means for each of the previous three monthly averages. This is called a moving average of the third order. The higher the order, the smoother the plot.

In [ ]:
fig, ax = plt.subplots()
sns.lineplot(data=df[['MONTHLY_AVG', 'MOVING_AVG']])
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
plt.xlabel('Datetime')
plt.ylabel('Avg. Energy Consumption (MW)')
plt.legend(labels=['Energy Consumption', '3-Month Moving Average'])
plt.title('ComEd Monthly Average Energy Consumption', fontsize=16)
plt.show()

A plot of the 3-month moving average shows a smoother timeseries with less fluctuations. This is a method typically used to estimate the trend of a dataset when performing decomposition.

## 10. Seasonal Trend Decomposition using Loess

Timeseries decomposition is a process that extracts the features for analysis. Seasonal Trend Decomposition using Loess (STL) is a specific type of decomposition that can estimate non-linear relationships. This method is robust to outliers and can handle any kind of seasonality within the data. The function will return three components of the timeseries:

* Season
* Trend
* Residual

A residual or remainder component is what is left over after extracting the season and the trend. A residual that shows strong elements of the trend or season reflects an incomplete decomposition. Residual points that deviate significantly can be classified as outliers.

In [ ]:
res = STL(df['MONTHLY_AVG'], period=12).fit()

# Create plots for each STL component
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1)

plt.suptitle('STL Decomposition')

sns.lineplot(data=df['MONTHLY_AVG'], ax=ax1, color='green')
ax1.set_title('ComEd Monthly Average Energy Consumption')
ax1.set_ylabel('')
ax1.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

sns.lineplot(data=res.trend, ax=ax2, color='navy')
ax2.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

sns.lineplot(data=res.seasonal, ax=ax3, color='red')
ax3.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

sns.lineplot(data=res.resid, ax=ax4, color='orange')
ax4.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

plt.tight_layout()
plt.show()

Here we see a seasonal component similar in pattern to our quarter and month plots. The smoothed trend displays a gradual drop in average energy usage from 2011 to 2016. The residual component is random and doesn't show significant signal from the trend or season.

## 11. Conclusion

We've employed various timeseries visualizations for analysis of our ComEd energy use case. Beginning with a standard time plot, we worked through lag plots and autocorrelations to differencing, seasonal plots, a moving average, and STL decomposition. The next phase in this timeseries exercise will be to explore forecasting methods.